# This notebook reproduces Table 5


## Install the following packages if necessary

In [2]:
#install.packages('xml2')
#install.packages('neat')
#install.packages('neat')

#if (!requireNamespace("BiocManager", quietly = TRUE))
#install.packages("BiocManager")

#BiocManager::install("biomaRt")


## Loading the packages


In [3]:

library('igraph')
library('biomaRt')
library('neat')
library('xml2')
#library('Oscope')
customlib='../R/CustomLibrary.R'


source(customlib)

workingdir='../casestudy/H1/results/'

beforefilt='../casestudy/H1/mergedH1.csv'
beforedata=read.csv(file=beforefilt, header= TRUE,row.names=1)
originalGenes=row.names(beforedata)



Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

Warning message:
“package ‘biomaRt’ was built under R version 3.6.1”

## loading the data and Graph creation


In [4]:

Oscoperunfile='mergedH1_N1000_g18_TFTrue_SummaryPartition.csv'
filea='../casestudy/H1/mergedH1_N1000_g18_TFTrue_SummaryPartition.csv'
#paste(workingdir,filelist[1],sep='')
ppp=strsplit(Oscoperunfile,split=".csv")
basefilea=ppp[[1]]
A=read.csv(file=filea,header=TRUE,sep=',')

G=graph_from_data_frame(A[,c(2,3,4)],directed =FALSE)


## Walktrap Community Extraction


In [5]:

# #####apply the KM Oscope Method
# MMM=get.adjacency(G,attr = 'cost')
# ##eps2=get.edge.attribute(G,'cost')
# ##min(eps2) ##this is greather then zero
# cut=-log10(min(MMM[which(as.matrix(MMM)!=0)]))
# MMM[which(as.matrix(MMM)==0)]=2*max(MMM)
# SimiMat=-log10(MMM)
# MyRes=list(SimiMat=as.matrix(SimiMat))
# KMmyRes <- OscopeKM(MyRes, cut = cut, maxK = 100, minSize = 1, maxSize = 1000)
# print(KMmyRes)
# 
###############WALKTRAP COMMUNITY Extraction
wt=walktrap.community(G,steps=4)
modularity(wt)
mwt=membership(wt)
sz=sizes(wt)
Mwt=as.matrix(membership(wt))
genes=row.names(Mwt)


[1] 0.5355635

## Retrieving ensembl information


In [6]:

###################################Retrieving ensembl information###############
caseE='ensembl_gene_id' #GENESYMBOL'
caseS="hgnc_symbol"###in case of mouse mgi_symbol
case=caseS
#speces="mmusculus_gene_ensembl"
speces="hsapiens_gene_ensembl"

#mart <- useDataset(speces, useMart("ensembl"))
#ensembl = useMart("ensembl",dataset="hsapiens_gene_ensembl")
mart = useMart("ensembl")
mart=useMart(biomart="ensembl", dataset="hsapiens_gene_ensembl")

CC=getBM(filters="go",
      values=c('GO:0007049'), mart=mart,
      attributes= c(caseE,caseS,"description"))
G_list <- getBM(filters= case, attributes= c(caseE,
                                            caseS,"description"),values=genes,mart= mart)
commonCCbefore=intersect(CC$hgnc_symbol,originalGenes)

commonCC=intersect(CC$hgnc_symbol,V(G)$name)


details=as.matrix(G_list)
colnames(Mwt)='CommunityID'
merMwt=Mwt#merge(details,Mwt,by.x=case,by.y=0,all.y=TRUE)
write.table(merMwt, file=paste(workingdir,basefilea,'Comm.txt',sep=''),sep='\t',quote=FALSE,row.names=FALSE)




In [7]:
merMwt

,CommunityID
ODF2L,24
FTH1,2
CFLAR,24
RPL23A,8
ASTN2,24
EIF3F,2
RPL41,8
PFN1,2
ORC4,24
KRI1,24


## merge communities with Cell Cycle related genes


In [8]:
CC=unique(CC$hgnc_symbol)
CC=as.data.frame(CC)
names(CC)='hgnc_symbol'
merMwt=data.frame(hgnc_symbol=row.names(merMwt),merMwt)

CCmerMwt=merge(CC,merMwt,by.x=caseS,by.y=caseS)
write.table(CCmerMwt, file=paste(workingdir,basefilea,'CC_Comm.txt',sep=''),sep='\t',quote=FALSE,row.names=FALSE)

vecComCC=unique(CCmerMwt$CommunityID)
sizeCCcom=vecComCC
EnrichmentTest=vecComCC
GlobalRatio=vecComCC
RelativeRatio=vecComCC


## Testing for communities CC enriched


In [9]:
######### Testing for communities CC enriched
white=length(unique(CCmerMwt$hgnc_symbol))#TotalCommonTargets
black=length(unique(merMwt$hgnc_symbol))-white
validation1=NULL
validation2=NULL
for (h in c(1:length(vecComCC))){
  
  ####neat testing for edges
  genlist1=unique(CCmerMwt$hgnc_symbol[which(CCmerMwt$CommunityID==vecComCC[h])])
  #myList[[length(myList)+1]] <- list(genlist1)
  genlist2=unique(merMwt$hgnc_symbol[which(merMwt$CommunityID==vecComCC[h])])
  alist = list('set 1' = genlist1)
  names(alist)=vecComCC[h]
  blist= list('set 2'=genlist2)
  test1 = neat(alist = alist, blist = alist, network = G,
               nettype = 'undirected', nodes = V(G)$name, alpha = 0.1)
  test2 = neat(alist = alist, blist = blist, network = G,
               nettype = 'undirected', nodes = V(G)$name, alpha = 0.1)
  validation1=rbind(validation1,print(test1))
  validation2=rbind(validation2,print(test2))
     
  ####hypergeometric testing for vertex
  sizeCCcom[h]=length(which(CCmerMwt$CommunityID==vecComCC[h]))
  x=sizeCCcom[h]#intersection
  k=length(which(merMwt$CommunityID==vecComCC[h]))#commsize
  EnrichmentTest[h]=1-phyper(x,white,black,k)
  GlobalRatio[h]=x/white
  RelativeRatio[h]=x/k
}
BHE=p.adjust(EnrichmentTest,method='BH')
outCC=cbind(vecComCC,sizeCCcom,GlobalRatio,RelativeRatio,EnrichmentTest,BHE)
colnames(outCC)=c('CC enriched Comm ID', 'size CC Overlapping','Global CC Ratio','Relative CC Ratio','CC Overlapping Significance','CC sig BH corr')
CCvalidation=validation1




## Wilcoxon Test on Community Significance


In [10]:
 
IdGE10=which(sz>1)
n10=length(IdGE10)
MatSummary=matrix('NA',n10,6)
colnames(MatSummary)=c("Min.","1st Qu.","Median","Mean","3rd Qu.","Max." )
NumberOfEdges=matrix('NA',n10,1)
CommDensity=matrix('NA',n10,1)
RelativeCommDensity=matrix('NA',n10,1)
PsigCom=matrix('NA',n10,1)
TotalEdges=ecount(G)
for (i in c(1:n10)){
  actCom=IdGE10[i]
  actNodes=which(Mwt==actCom)
  actNumVertex=length(actNodes)
  subG=induced_subgraph(G,actNodes)
  actEdge=E(subG)
  actNumEdges=ecount(subG)
  NumberOfEdges[i]=actNumEdges
  CommDensity[i]=actNumEdges/TotalEdges
  RelativeCommDensity[i]=2*actNumEdges/(actNumVertex^2-actNumVertex)
  
  actCost=get.edge.attribute(subG,'cost',index=actEdge)
  vsumAct=as.vector(summary(actCost))
  MatSummary[i,]=vsumAct
  res=community.significance.test(G, actNodes)
  PsigCom[i]=res$p.value
}


## Selection of Communities with Wilcoxon Significance<0.01


In [11]:

sizes10=sz[IdGE10]
sizes10=data.frame(Community=names(sizes10),NumberOfNodes= as.vector(sizes10))

Report10=data.frame(sizes10,NumberOfEdges=NumberOfEdges,Significance=PsigCom,Density=CommDensity,RelativeDensity=RelativeCommDensity,MatSummary)
Report10sig=Report10[which(as.numeric(as.vector(Report10$Significance))<0.01),]
write.table(Report10sig, file=paste(workingdir,'Report1sig','.txt',sep=''),sep='\t',quote=FALSE,row.names=FALSE,col.names=TRUE)

####Merging the results and selecting the relevant information
prep1=data.frame(sizes10,NumberOfEdges=NumberOfEdges,Significance=PsigCom,Density=CommDensity,RelativeDensity=RelativeCommDensity)
prep1 = prep1[which(as.numeric(as.vector(prep1$Significance))<0.01),]


valCC=merge(prep1, outCC , by.x='Community',by.y='CC enriched Comm ID',all.x=TRUE)

valCC=merge(valCC,CCvalidation,by.x='Community',by.y='A',all.x=TRUE)


Table5=data.frame(Size=as.vector(valCC$NumberOfNodes),CC=as.vector(valCC$'size CC Overlapping'),Density=as.vector(valCC$Density),Realative_density=as.vector(valCC$RelativeDensity),Hypergeometic=as.vector(valCC$'CC sig BH corr'), Significance=valCC$Significance, Neat=as.vector(valCC$adjusted_p))

Table5=Table5[order(Table5$Size),]

write.table(Table5, file=paste(workingdir,'Table5','.txt',sep=''),sep='\t',quote=FALSE,row.names=FALSE,col.names=TRUE)


In [12]:
Table5

,Size,CC,Density,Realative_density,Hypergeometic,Significance,Neat
,<int>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>
4,4,1,0.000593237097093138,0.5,0.03487963,0.00885303290368331,9.999011e-01
3,6,NA,0.00098872849515523,0.333333333333333,NA,0.00376315758322894,NA
5,77,32,0.0864148704765671,0.149350649350649,0.00000000,1.73812197922636e-09,3.071383e-53
1,155,7,0.144156614593633,0.0610808546292417,0.84405341,3.51440629797138e-12,4.785181e-01
2,188,11,0.102630017797113,0.0295255432927523,0.65060900,2.49482540784068e-10,1.642181e-01
6,234,16,0.149495748467471,0.0277319247276329,0.44513881,1.24093440617898e-13,4.480050e-01
